**Important note!** Before you turn in this lab notebook, make sure everything runs as expected:

- First, **restart the kernel** -- in the menubar, select Kernel$\rightarrow$Restart.
- Then **run all cells** -- in the menubar, select Cell$\rightarrow$Run All.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE."

# Lesson 0: SQLite

The de facto language for managing relational databases is the Structured Query Language, or SQL ("sequel").

Many commerical and open-source relational data management systems (RDBMS) support SQL. The one we will consider in this class is the simplest, called [sqlite3](https://www.sqlite.org/). It stores the database in a simple file and can be run in a "standalone" mode from the command-line. However, we will, naturally, [invoke it from Python](https://docs.python.org/3/library/sqlite3.html). But all of the basic techniques apply to any commercial SQL backend.

With a little luck, you _might_ by the end of this class understand this [xkcd comic on SQL injection attacks](http://xkcd.com/327).

## Getting started

In Python, you _connect_ to an `sqlite3` database by creating a _connection object_.

**Exercise 0** (ungraded). Run this code cell to get started.

In [ ]:
import sqlite3 as db

# Connect to a database (or create one if it doesn't exist)
conn = db.connect('example.db')

The `sqlite` engine maintains a database as a file; in this example, the name of that file is `example.db`.

> **Important usage note!** If the named file does **not** yet exist, this code creates it. However, if the database has been created before, this same code will open it. This fact can be important when you are debugging. For example, if your code depends on the database not existing initially, then you may need to remove the file first.

You issue commands to the database through an object called a _cursor_.

In [ ]:
# Create a 'cursor' for executing commands
c = conn.cursor()

A cursor tracks the current state of the database, and you will mostly be using the cursor to issue commands that modify or query the database.

## Tables and Basic Queries

The central object of a relational database is a _table_. It's identical to what you called a "tibble" in the tidy data lab: observations as rows, variables as columns. In the relational database world, we sometimes refer to as _items_ or _records_ and columns as _attributes_. We'll use all of these terms interchangeably in this course.

Let's look at a concrete example. Suppose we wish to maintain a database of Georgia Tech students, whose attributes are their names and Georgia Tech-issued ID numbers. You might start by creating a table named `Students` to hold this data. You can create the table using the command, [`create table`](https://www.sqlite.org/lang_createtable.html).

> Note: If you try to create a table that already exists, it will **fail**. If you are trying to carry out these exercises from scratch, you may need to remove any existing `example.db` file or destroy any existing table; you can do the latter with the SQL command, `drop table if exists Students`.

In [ ]:
c.execute("create table Students (gtid integer, name text)")

To populate the table with items, you can use the command, [`insert into`](https://www.sqlite.org/lang_insert.html).

In [ ]:
c.execute("insert into Students values (123, 'Vuduc')")
c.execute("insert into Students values (456, 'Chau')")
c.execute("insert into Students values (381, 'Bader')")
c.execute("insert into Students values (991, 'Sokol')")

**Commitment issues.** The commands above modify the database. However, these are temporary modifications and aren't actually saved to the databases until you say so. (_Aside:_ Why would you want such behavior?) The way to do that is to issue a _commit_ operation from the _connection_ object.

> There are some subtleties related to when you actually need to commit, since the SQLite database engine does commit at certain points as discussed [here](https://stackoverflow.com/questions/13642956/commit-behavior-and-atomicity-in-python-sqlite3-module). However, it's probably simpler if you remember to encode commits when you intend for them to take effect.

In [ ]:
conn.commit()

Another common operation is to perform a bunch of insertions into a table from a list of tuples. In this case, you can use `executemany()`.

In [ ]:
# An important (and secure!) idiom
more_students = [(723, 'Rozga'),
                 (882, 'Zha'),
                 (401, 'Park'),
                 (377, 'Vetter'),
                 (904, 'Brown')]

c.executemany('insert into Students values (?, ?)', more_students)
conn.commit()

Given a table, the most common operation is a _query_, which asks for some subset or transformation of the data. The simplest kind of query is called a [`select`](https://www.sqlite.org/lang_select.html).

The following example selects all rows (items) from the `Students` table.

In [ ]:
c.execute("select * from Students")
results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Students:\n", results)

**Exercise 1** (2 points). Suppose we wish to maintain a second table, called `Takes`, which records classes that students have taken and the grades they earn.

In particular, each row of `Takes` stores a student by his/her GT ID, the course he/she took, and the grade he/she earned. More formally, suppose this table is defined as follows:

In [ ]:
# Run this cell
c.execute('drop table if exists Takes')
c.execute('create table Takes (gtid integer, course text, grade real)')

Write a command to insert the following records into the `Takes` table.

* Vuduc: CSE 6040 - A (4.0), ISYE 6644 - B (3.0), MGMT 8803 - D (1.0)
* Sokol: CSE 6040 - A (4.0), ISYE 6740 - A (4.0)
* Chau: CSE 6040 - A (4.0), CSE 6740 - C (2.0), MGMT 8803 - B (3.0)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# Displays the results of your code
c.execute('select * from Takes')
results = c.fetchall()
print("Your results:", len(results), "\nThe entries of Takes:", results)

In [ ]:
# Test cell: `insert_many__test`

# Close the database and reopen it
conn.close()
conn = db.connect('example.db')
c = conn.cursor()
c.execute('select * from Takes')
results = c.fetchall()

if len(results) == 0:
    print("*** No matching records. Did you remember to commit the results? ***")
assert len(results) == 8, "The `Takes` table has {} when it should have {}.".format(len(results), 8)

assert (123, 'CSE 6040', 4.0) in results
assert (123, 'ISYE 6644', 3.0) in results
assert (123, 'MGMT 8803', 1.0) in results
assert (991, 'CSE 6040', 4.0) in results
assert (991, 'ISYE 6740', 4.0) in results
assert (456, 'CSE 6040', 4.0) in results
assert (456, "CSE 6740", 2.0) in results
assert (456, "MGMT 8803", 3.0) in results

print("\n(Passed.)")

# Lesson 1: Join queries

The main type of query that combines information from multiple tables is the _join query_. Recall from our discussion of tibbles these four types:

- `inner-join(A, B)`: Keep rows of `A` and `B` only where `A` and `B` match
- `outer-join(A, B)`: Keep all rows of `A` and `B`, but merge matching rows and fill in missing values with some default (`NaN` in Pandas, `NULL` in SQL)
- `left-join(A, B)`: Keep all rows of `A` but only merge matches from `B`.
- `right-join(A, B)`: Keep all rows of `B` but only merge matches from `A`.

If you are a visual person, see [this page](https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins) for illustrations of the different join types.

In SQL, you can use the `where` clause of a `select` statement to specify how to match rows from the tables being joined. For example, recall that the `Takes` table stores classes taken by each student. However, these classes are recorded by a student's GT ID. Suppose we want a report where we want each student's name rather than his/her ID. We can get the matching name from the `Students` table. Here is a query to accomplish this matching:

In [ ]:
# See all (name, course, grade) tuples
query = '''
    select Students.name, Takes.course, Takes.grade
        from Students, Takes
        where Students.gtid=Takes.gtid
'''

for match in c.execute(query): # Note this alternative idiom for iterating over query results
    print(match)

**Exercise 2** (2 points). Define a query to select only the names and grades of students _who took CSE 6040_. The code below will execute your query and store the results in a list `results1` of tuples, where each tuple is a `(name, grade)` pair; thus, you should structure your query to match this format.

In [ ]:
# Define `query` with your query:
# YOUR CODE HERE
raise NotImplementedError()

c.execute(query)
results1 = c.fetchall()
results1

In [ ]:
# Test cell: `join1__test`

print ("Your results:", results1)

assert type(results1) is list
assert len(results1) == 3, "Your query produced {} results instead of {}.".format(len(results1), 3)

assert set(results1) == {('Vuduc', 4.0), ('Sokol', 4.0), ('Chau', 4.0)}

print("\n(Passed.)")

For contrast, let's do a quick exercise that executes a [left join](http://www.sqlitetutorial.net/sqlite-left-join/).

**Exercise 3** (2 points). Execute a left join that uses `Students` as the left table, `Takes` as the right table, and selects a student's name and course grade. Write your query as a string variable named `query`, which the subsequent code will execute.

In [ ]:
# Define `query` string here:
# YOUR CODE HERE
raise NotImplementedError()

# Executes your `query` string:
c.execute(query)
matches = c.fetchall()
for i, match in enumerate(matches):
    print(i, "->", match)

In [ ]:
# Test cell: `left_join_test`

assert set(matches) == {('Vuduc', 4.0), ('Chau', 2.0), ('Park', None), ('Vuduc', 1.0), ('Chau', 3.0), ('Zha', None), ('Brown', None), ('Vetter', None), ('Vuduc', 3.0), ('Bader', None), ('Rozga', None), ('Chau', 4.0), ('Sokol', 4.0)}
print("\n(Passed!)")

## Aggregations

Another common style of query is an _aggregation_, which is a summary of information across multiple records, rather than the raw records themselves.

For instance, suppose we want to compute the GPA for each unique GT ID from the `Takes` table. Here is a query that does it:

In [ ]:
query = '''
    select gtid, avg(grade)
        from Takes 
        group by gtid
'''

for match in c.execute(query):
    print(match)

Some other useful SQL aggregators include `min`, `max`, `sum`, and `count`.

## Cleanup

As one final bit of information, it's good practice to shutdown the cursor and connection, the same way you close files.

In [ ]:
c.close()
conn.close()

**What next?** It's now a good time to look at a different tutorial which reviews this material and introduces some additional topics: [A thorough guide to SQLite database operations in Python](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html).